# Accessing the Scalar Adder via PYNQ
In this demo, we show how to access the simple scalar adder via PYNQ.

Before running this demo, you should have followed the instructions in [getting started demo](https://sdrangan.github.io/hwdesign/scalar_fun/) to build the scalar adder IP, exported it to a Vivado project and build the overlay.

We first load some key packages.

In [8]:
import numpy as np

We can access the overlay that we have created with the pynq command.

In [10]:
from pynq import Overlay
overlay = Overlay("../scalar_fun_pynqz2/overlay/scalar_fun_pynqz2.bit")

We can then print out information on the overlay.

In [11]:
overlay?

Type:            Overlay
String form:     <pynq.overlay.Overlay object at 0xafde2e30>
File:            /usr/local/lib/python3.6/dist-packages/pynq/overlay.py
Docstring:      
Default documentation for overlay ../scalar_fun_pynqz2/overlay/scalar_fun_pynqz2.bit. The following
attributes are available on this overlay:

IP Blocks
----------
simp_fun_0           : pynq.overlay.DefaultIP

Hierarchies
-----------
None

Interrupts
----------
None

GPIO Outputs
------------
None

Memories
------------
processing_system7_0 : Memory
Class docstring:
This class keeps track of a single bitstream's state and contents.

The overlay class holds the state of the bitstream and enables run-time
protection of bindlings.

Our definition of overlay is: "post-bitstream configurable design".
Hence, this class must expose configurability through content discovery
and runtime protection.

The overlay class exposes the IP and hierarchies as attributes in the
overlay. If no other drivers are available the `Defaul

We see that the overlay has two IPs:  The Zynq and the Adder.  We access the IP and gets its register map.  We see that the register map has the inputs `a` and `b`, the output `c`, and some other control messages.

In [12]:
ip = overlay.simp_fun_0
ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  a = Register(a=0),
  b = Register(b=0),
  c = Register(c=0),
  c_ctrl = Register(c_ap_vld=0, RESERVED=0)
}

The following code indicates a simple way to use the IP:
*  Set the registers `a` and `b`
*  Start operation 
*  Wait until completion by polling the control register
*  Read the output `c`.

Later, we will show how to avoid the polling with interrupts.

In [17]:
from time import sleep


for i in range(5):
    # Input parameters 
    a = np.random.randint(low=0, high=128)
    b = np.random.randint(low=0, high=128)

    # Sets the two inputs
    ip.register_map.a = a
    ip.register_map.b = b

    # Start the execution
    ip.register_map.CTRL.AP_START = 1

    # Wait for the execution to complete
    # Right now we wait a bit between polling events
    # to prevent overflowing the bus
    while not ip.register_map.CTRL.AP_IDLE:
        sleep(0.001) 

    # Get the value
    c = int(ip.register_map.c)
    
    # Print the expected value
    c_exp = a * b
    if (c == c_exp):
        res = 'pass'
    else:
        res = 'fail'
            
    print('a: %5d b: %5d c: %5d %5d %s' % (a,b,c,c_exp, res))

a:    21 b:    34 c:   714   714 pass
a:   113 b:   117 c: 13221 13221 pass
a:    86 b:    27 c:  2322  2322 pass
a:   120 b:    52 c:  6240  6240 pass
a:    47 b:    34 c:  1598  1598 pass
